In [5]:
import numpy as np 
import pandas as pd 
import re
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


In [7]:
df = pd.read_csv('../quora-question-pairs/train.csv')

In [8]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [9]:
MAX_NB_WORDS = 200000
tokenizer = Tokenizer(num_words = MAX_NB_WORDS)
tokenizer.fit_on_texts(list(df['question1'].values.astype(str))+list(df['question2'].values.astype(str)))

In [13]:
word_index = tokenizer.word_index
embedding_index = {}
with open('../quora-question-pairs/glove.840B.300d.txt','r') as f:
    for line in f:
        values = line.split()
        vectors = np.asarray(values[-300:], dtype='float32')
        word = ''.join(values[:-300])  
        embedding_index[word] = vectors
    f.close()

embedding_matrix = np.random.random((len(word_index)+1, 300))
for word, i in word_index.items():
    if embedding_index.get(word) is not None:
        embedding_matrix[i] = embedding_index.get(word)

In [ ]:
def find_embedding(X_train_q1):
    l = len(X_train_q1)
    X_train_q1_embeddings = np.zeros((0))
    for j in range(l):
      q11=X_train_q1[j]
      q1 = ''.join([m if ord(m) < 128 else ' ' for m in q11])    #remove non ascii
    try:
            str1=q1.split(" ")
            l1=len(str1)
            f1=np.array((300))
            for i in range(l1):
                word = str1[i]
                f1 = np.add(f1,embedding_matrix[word_index[word]])
            X_train_q1_embeddings=np.append(X_train_q1_embeddings,f1)
    except:
            e = sys.exc_info()
            print(e)
            continue
    X_train_q1_embeddings = X_train_q1_embeddings.reshape(l,300)
    return  X_train_q1_embeddings

In [ ]:
from sklearn.model_selection import train_test_split
y = df['is_duplicate']
X_train,X_test,y_train,y_test = train_test_split(df,y, test_size = float(3/10), random_state =0)
X_valid,X_test,y_valid,y_test = train_test_split(X_test,y_test, test_size = float(1/3), random_state = 0)

In [ ]:
X_train_q1 = X_train.question1.values
X_train_q2 = X_train.question2.values
X_test_q1 = X_test.question1.values
X_test_q2 = X_test.question2.values

In [ ]:
import sys
X_train_q1_embeddings = np.zeros((len(X_train_q1),300))
X_train_q1_embeddings = find_embedding(X_train_q1)
X_train_q2_embeddings = np.zeros((len(X_train_q2),300))
X_train_q2_embeddings = find_embedding(X_train_q2)

In [ ]:
def concatenation(X1,X2):
    l=len(X1)
    X_train_embeddings = np.zeros((l, 300))
    for i in range(l):
        X_train_embeddings[i] = np.concatenate((X1[i], X2[i]), axis=0)
    return X_train_embeddings

In [ ]:
X_train_final = concatenation(X_train_q1_embeddings,X_train_q2_embeddings)

In [ ]:
X_test_q1_embeddings = np.zeros((len(X_test_q1),300))
X_test_q1_embeddings = find_embedding(X_test_q1)
X_test_q2_embeddings = np.zeros((len(X_test_q2),300))
X_test_q2_embeddings = find_embedding(X_test_q2)
X_test_final = concatenation(X_test_q1_embeddings,X_test_q2_embeddings)

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(solver='adam',alpha=1e-5,hidden_layer_sizes=(100),random_state=1)
ml=mlp.fit(X_train_final,y_train)
y_pred = mlp.predict(X_test_final)


In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,y_pred)
print(accuracy)

In [ ]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test,y_pred)
print(f1)